In [1]:
# !pip install osmnx networkx pandas numpy matplotlib geopy lxml pickle5 folium keplergl

import os
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import matplotlib.pyplot as plt
import folium
from folium import plugins
import pickle
# from keplergl import KeplerGl

from networkx.classes import MultiDiGraph

os.makedirs('map_data', exist_ok=True)

##### 1. Загрузка и очистка

In [2]:
def load_graph(location: str) -> MultiDiGraph:
    print(f"Загрузка: {location}...")

    G = ox.graph_from_place(
        location,
        network_type='drive',
        simplify=True
    )

    print(f"Загружено: {location}")

    G.remove_nodes_from(list(nx.isolates(G)))

    largest = max(nx.strongly_connected_components(G), key=len)
    G = G.subgraph(largest).copy()

    print(f"После очистки: {G.number_of_nodes()} узлов, {G.number_of_edges()} рёбер")
    return G

In [3]:
# Загрузить все 3 графа через функцию
graphs = {}

# Москва
G_moscow = load_graph("Moscow, Russia")
graphs['moscow'] = G_moscow

# Крупный район(САО)
G_district_large = load_graph("Северный административный округ, Москва")
graphs['district_large'] = G_district_large

# Малый район(Замоскворечье)
G_district_small = load_graph("Замоскворечье, Москва")
graphs['district_small'] = G_district_small

Загрузка: Moscow, Russia...
Загружено: Moscow, Russia
После очистки: 26955 узлов, 58206 рёбер
Загрузка: Северный административный округ, Москва...
Загружено: Северный административный округ, Москва
После очистки: 1587 узлов, 3318 рёбер
Загрузка: Замоскворечье, Москва...
Загружено: Замоскворечье, Москва
После очистки: 178 узлов, 372 рёбер


In [4]:
Gs = {
	'moscow': G_moscow,
	'sao': G_district_large,
	'zmr': G_district_small
}

##### 2. Функциональные веса

In [5]:
speeds = {
    'motorway': 100,
    'trunk': 90,
    'primary': 70,
    'secondary': 60,
    'tertiary': 50,
    'unclassified': 40,
    'residential': 30,
    'living_street': 20,
}
for name, G in Gs.items():
	for u, v, k, data in G.edges(keys=True, data=True):
			highway = data.get('highway', 'unclassified')
			if isinstance(highway, list):
					highway = highway[0]

			# реальная скорость 70% от максимальной
			speed_kph = speeds.get(highway, 40) * 0.7

			length_km = data.get('length', 100) / 1000

			# в минутах
			travel_time = (length_km / speed_kph) * 60

			data['travel_time'] = travel_time
			data['speed_kph'] = speed_kph

##### 3. Набор O-D (Отправление-Назначение) пар

In [14]:
moscow_center = (55.7558, 37.6173)

for name, G in Gs.items():
	print(f"{name}:")

	if name == 'moscow':
		center_radius = 8000
	elif name == 'sao':
		center_radius = 12000
	elif name == 'zmr':
		center_radius = 2500

	center_nodes = []
	periphery_nodes = []

	for node in G.nodes():
			lat = G.nodes[node]['y']
			lon = G.nodes[node]['x']
			dist = geodesic(moscow_center, (lat, lon)).meters

			if dist < center_radius:
					center_nodes.append(node)
			else:
					periphery_nodes.append(node)

	print(f"Узлов в центральном районе: {len(center_nodes)}")
	print(f"Узлов на периферии: {len(periphery_nodes)}")

	np.random.seed(1984)
	n_center = min(10, len(center_nodes))
	n_periphery = min(5000, len(periphery_nodes))

	center_sample = np.random.choice(center_nodes, n_center, replace=False)
	periphery_sample = np.random.choice(periphery_nodes, n_periphery, replace=False)

	print(f"Выбрано для O-D пар:")
	print(f"    В центральном районе: {n_center}")
	print(f"    На периферии: {n_periphery}")

	print(f"Создание O-D пар...")

	od_pairs = []
	for _ in range(5000):
			origin = np.random.choice(center_sample)
			destination = np.random.choice(periphery_sample)
			if origin != destination:
					od_pairs.append({
							'node_id_origin': origin,
							'node_id_dest': destination
					})

	df_od = pd.DataFrame(od_pairs)
	print(f"Создано {len(df_od)} O-D пар")
	df_od.to_csv(f"./map_data/{name}_od_pairs.csv")
	print(f"Результат сохранён в ./map_data/{name}_od_pairs.csv")

	print()

moscow:
Узлов в центральном районе: 6765
Узлов на периферии: 20190
Выбрано для O-D пар:
    В центральном районе: 10
    На периферии: 5000
Создание O-D пар...
Создано 5000 O-D пар
Результат сохранён в ./map_data/moscow_od_pairs.csv

sao:
Узлов в центральном районе: 1036
Узлов на периферии: 551
Выбрано для O-D пар:
    В центральном районе: 10
    На периферии: 551
Создание O-D пар...
Создано 5000 O-D пар
Результат сохранён в ./map_data/sao_od_pairs.csv

zmr:
Узлов в центральном районе: 64
Узлов на периферии: 114
Выбрано для O-D пар:
    В центральном районе: 10
    На периферии: 114
Создание O-D пар...
Создано 5000 O-D пар
Результат сохранён в ./map_data/zmr_od_pairs.csv



##### 4. Сохранение графа


In [15]:
# заюзал pickle, тк graphml плохо сериализует
for name, G in Gs.items():
	filename = f'./map_data/{name}.pkl'
	print(f"Сохранение {name} в {filename}   ...")
	with open(filename, 'wb') as f:
			pickle.dump(G, f)
	print(f"Сохранено {filename}")

Сохранение moscow в ./map_data/moscow.pkl   ...
Сохранено ./map_data/moscow.pkl
Сохранение sao в ./map_data/sao.pkl   ...
Сохранено ./map_data/sao.pkl
Сохранение zmr в ./map_data/zmr.pkl   ...
Сохранено ./map_data/zmr.pkl


In [17]:
print("Проверка чтения...")
for name, G in Gs.items():
	print(f"{name:}")
	filename = f'./map_data/{name}.pkl'
	with open(filename, 'rb') as f:
			G_check = pickle.load(f)
	print(f"Успешно прочитано: {G_check.number_of_nodes()} узлов, {G_check.number_of_edges()} рёбер")
	assert G_check.number_of_nodes() == G.number_of_nodes()
	assert G_check.number_of_edges() == G.number_of_edges()
	u, v, k, data = list(G_check.edges(keys=True, data=True))[0]
	print(data)
	print()

print(f"Проверка пройдена!")

Проверка чтения...
moscow
Успешно прочитано: 26955 узлов, 58206 рёбер
{'osmid': [541518490, 541518491, 541334132], 'highway': 'secondary_link', 'lanes': '1', 'maxspeed': '30', 'oneway': True, 'reversed': False, 'length': np.float64(639.7110252830249), 'bridge': 'yes', 'geometry': <LINESTRING (37.382 55.522, 37.38 55.521, 37.379 55.521, 37.379 55.52, 37.37...>, 'travel_time': np.float64(1.370809339892196), 'speed_kph': 28.0}

sao
Успешно прочитано: 1587 узлов, 3318 рёбер
{'osmid': 230027853, 'highway': 'tertiary', 'lanes': '6', 'maxspeed': 'RU:urban', 'name': 'Дмитровское шоссе', 'oneway': False, 'reversed': False, 'length': np.float64(38.561263922511536), 'travel_time': np.float64(0.06610502386716263), 'speed_kph': 35.0}

zmr
Успешно прочитано: 178 узлов, 372 рёбер
{'osmid': 157320219, 'highway': 'unclassified', 'lanes': '2', 'maxspeed': 'RU:urban', 'name': '3-й Люсиновский переулок', 'oneway': False, 'reversed': True, 'length': np.float64(243.6692548553865), 'geometry': <LINESTRING (3

##### 5. Базовый анализ

In [14]:
# LCC fraction
lcc_nodes = max(nx.weakly_connected_components(G), key=len)
lcc_fraction = len(lcc_nodes) / G.number_of_nodes()

# Плотность
density = nx.density(G.to_undirected())

# Средняя степень
degrees = [G.degree(n) for n in G.nodes()]
avg_degree = np.mean(degrees)

# travel_time по всем рёбрам
travel_times = []
for u, v, k, data in G.edges(keys=True, data=True):
    if 'travel_time' in data:
        travel_times.append(float(data['travel_time']))

avg_travel_time = np.mean(travel_times)
max_travel_time = np.max(travel_times)
min_travel_time = np.min(travel_times)
median_travel_time = np.median(travel_times)

stats_output = f"""Узлов: {G.number_of_nodes():,}
Рёбер: {G.number_of_edges():,}
LCC fraction: {lcc_fraction:.4f}
Плотность: {density:.6f}
Средняя степень: {avg_degree:.2f}
Travel_time среднее: {avg_travel_time:.2f} мин
Travel_time макс: {max_travel_time:.2f} мин
Travel_time медиана: {median_travel_time:.2f} мин
O-D пар: {len(df_od)}"""

# Сохрани в файл
with open('./map_data/base_stats.txt', 'w') as f:
    f.write(stats_output)

print(stats_output)

Узлов: 26,948
Рёбер: 58,189
LCC fraction: 1.0000
Плотность: 0.000107
Средняя степень: 4.32
Travel_time среднее: 0.48 мин
Travel_time макс: 10.72 мин
Travel_time медиана: 0.30 мин
O-D пар: 5000


##### 5. Базовый анализ

In [16]:
# тут вайбкод!

# Центр Москвы
moscow_center = (55.7558, 37.6173)

# Создай карту
m = folium.Map(
    location=moscow_center,
    zoom_start=11,
    tiles='OpenStreetMap'
)

# Добавь слой со всеми узлами сети (выборка для скорости)
print("\n📍 Добавляю узлы сети...")
sample_nodes = list(G.nodes())[:min(1000, G.number_of_nodes())]

for node in sample_nodes:
    lat = G.nodes[node]['y']
    lon = G.nodes[node]['x']

    folium.CircleMarker(
        location=(lat, lon),
        radius=2,
        color='gray',
        fill=True,
        fillOpacity=0.3,
        weight=0.5,
        popup=f"Node {node}"
    ).add_to(m)

print(f"   ✓ Добавлено {len(sample_nodes)} узлов (выборка)")

# Добавь O-D пары
print("📌 Добавляю O-D пары...")
sample_od = df_od.sample(min(500, len(df_od)), random_state=42)

for idx, row in sample_od.iterrows():
    o_node = row['node_id_origin']
    d_node = row['node_id_dest']

    if o_node in G.nodes() and d_node in G.nodes():
        o_lat, o_lon = G.nodes[o_node]['y'], G.nodes[o_node]['x']
        d_lat, d_lon = G.nodes[d_node]['y'], G.nodes[d_node]['x']

        # Origin (зелёный)
        folium.CircleMarker(
            location=(o_lat, o_lon),
            radius=5,
            color='green',
            fill=True,
            fillOpacity=0.7,
            weight=2,
            popup=f"Origin {o_node}"
        ).add_to(m)

        # Destination (синий)
        folium.CircleMarker(
            location=(d_lat, d_lon),
            radius=5,
            color='blue',
            fill=True,
            fillOpacity=0.7,
            weight=2,
            popup=f"Dest {d_node}"
        ).add_to(m)

        # Линия между ними (красная)
        folium.PolyLine(
            locations=[(o_lat, o_lon), (d_lat, d_lon)],
            color='red',
            weight=1,
            opacity=0.5
        ).add_to(m)

print(f"   ✓ Добавлено {len(sample_od)} O-D маршрутов")

# Добавь легенду
legend_html = '''
<div style="position: fixed;
     bottom: 50px; right: 50px; width: 200px; height: 150px;
     background-color: white; border:2px solid grey; z-index:9999;
     font-size:14px; padding: 10px">

<p><b>Легенда</b></p>
<p><i class="fa fa-circle" style="color:gray"></i> Узлы сети</p>
<p><i class="fa fa-circle" style="color:green"></i> Origin (центр)</p>
<p><i class="fa fa-circle" style="color:blue"></i> Destination (периферия)</p>
<p style="color:red">━ O-D маршруты</p>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Сохрани
map_file = 'map_data/moscow_network_od_pairs.html'
m.save(map_file)
print(f"\n✓ Карта сохранена: {map_file}")
print(f"   Открой в браузере для просмотра")

# Статистика карты
print(f"\n📊 Статистика карты:")
print(f"   - Узлов на карте: {len(sample_nodes)}")
print(f"   - O-D пар на карте: {len(sample_od)}")
print(f"   - Всего O-D пар в данных: {len(df_od)}")
print(f"   - Зум: 11 (хороший для всей Москвы)")


📍 Добавляю узлы сети...
   ✓ Добавлено 1000 узлов (выборка)
📌 Добавляю O-D пары...
   ✓ Добавлено 500 O-D маршрутов

✓ Карта сохранена: map_data/moscow_network_od_pairs.html
   Открой в браузере для просмотра

📊 Статистика карты:
   - Узлов на карте: 1000
   - O-D пар на карте: 500
   - Всего O-D пар в данных: 5000
   - Зум: 11 (хороший для всей Москвы)


In [18]:
# У меня на винде kepler не ставится, не тестил

od_map_data = []
for idx, row in df_od.sample(1000).iterrows():
    o_node = row['node_id_origin']
    d_node = row['node_id_dest']

    if o_node in G.nodes() and d_node in G.nodes():
        od_map_data.append({
            'latitude': G.nodes[o_node]['y'],
            'longitude': G.nodes[o_node]['x'],
            'type': 'origin'
        })
        od_map_data.append({
            'latitude': G.nodes[d_node]['y'],
            'longitude': G.nodes[d_node]['x'],
            'type': 'destination'
        })

df_kepler = pd.DataFrame(od_map_data)

# map_kepler = KeplerGl(height=400)
# map_kepler.add_data(data=df_kepler, name='od_pairs')
# map_kepler.save_to_html(file_name='map_data/moscow_kepler.html')